In [ ]:
import sys
sys.path.append('..')

In [ ]:
from copy import deepcopy
from random import shuffle, choice
from library.solution import Solution
from library.algorithms.hill_climbing import hill_climbing

In previous notebooks, we defined the `Solution` class as an **abstract class** with methods that must be implemented by subclasses, depending on the specific optimization problem. While the implementation of solutions depend on the problem, all solutions share common principles: they require a **representation**, a **fitness function**, and a method for **random initialization**.

By extending this class, we can define solution classes specific to different optimization problems. For example, we created the `IntBinSolution` class to represent solutions for the IntBin optimization problem.

We then applied the Hill Climbing algorithm to the IntBin problem by further extending `IntBinSolution` to implement the `get_neighbors()` method, which is essential for Hill Climbing algorithm. To do this, we created a new class, `IntBinHillClimbingSolution`.

Today, we'll use Hill Climbing to solve two new problems: the Traveling Salesperson Problem (TSP) and the Knapsack Problem (KS).

## Traveling Salesperson Problem

**Description:** The Traveling Salesperson Problem (TSP) is the challenge of finding the shortest possible route that starts in a given city, visits each of the remaining N-1 cities exactly once, and returns to the starting city.

**Search space:** All possible permutations of city visit orders, forming valid round-trip routes.

**Representation:** List of city indexes that compose the route

**Fitness function:** f(x) = Total distance traveled, computed as the sum of distances between consecutive cities in the route.

**Goal:** Minimize f(x).

Let's begin by implementing the `TSPSolution` class, which inherits from `Solution`. As a result, it must implement the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Traveling Salesperson Problem (TSP) and does not include any implementation related to the optimization algorithm that will be used to solve it.

![TSP Solution](images/tsp-solution.png)

In [ ]:
from library.problems.data.tsp_data import distance_matrix

#TODO: Implement TSPSolution class

Let's test it

In [ ]:
solution = TSPSolution()

print('Random solution:', solution)
print('Fitness:', solution.fitness())

### Solving TSP with Hill Climbing

To use Hill Climbing to solve TSP we need to define a `TSPHillClimbingSolution` class that implements the `get_neighbors()` method. We also need to ensure that this function returns a list of solutions that also implement the `get_neighbors()` method, therefore, return a list of solutions of type `TSPHillClimbingSolution` too.

A TSP neighbor solution is obtained by swapping the positions of two consecutive cities in the route.

![TSP Hill Climbing Solution](images/tsp-hillclimbing-solution.png)

In [ ]:
#TODO: Implement TSPSHillClimbingSolution class

Let's test it

In [ ]:
solution = TSPHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [ ]:
#TODO: Apply hill climbing to TSP

The implementations of `TSPSolution` and `TSPHillClimbingSolution` can be found in `library/problems/tsp.py`

## Knapsack Problem

**Description:** The Knapsack Problem involves selecting a subset of N items, each with a given value and weight, to pack into a container with a fixed capacity. If the total weight of selected items exceeds the capacity, the solution is invalid. The goal is to maximize the total value of items while ensuring they fit within the container's constraints.

**Search space:** All possible subsets of items that can be placed in the knapsack.

**Representation:** Binary string of length N (number of items), where 1 indicates the item is included in the knapsack and 0 indicates the item is excluded.

**Fitness function:** f(x)= Total value inside the knapsack. If the total size exceeds the knapsack's capacity, the solution is invalid and assigned a fitness of -inf.

**Goal:** Maximize f(x).

Similarly to what we've done for TSP, let's begin by implementing the `KSSolution` class, which inherits from `Solution` and implementes the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Knapsack problem (KS) and does not include any implementation related to the optimization algorithm that will be used to solve it.

In [ ]:
from library.problems.data.ks_data import values, weights, capacity

#TODO: Implement KSSolution class

Let's test it

In [ ]:
solution = KSSolution()

print(solution)
print(solution.fitness())

### Solving KS with Hill Climbing

A neighbor solution is obtained by flipping a single bit, meaning adding one item to the knapsack, or removing one item from the knapsack.

Let's create the `KSHillClimbingSolution` that inherits from `KSSolution` and implements the `get_neighbors` method.


In [ ]:
#TODO: Implement KSHillClimbingSolution class

Let's test it

In [ ]:
solution = KSHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [ ]:
#TODO: Apply hill climbing to KS problem

The implementations of `KSSolution` and `KSHillClimbingSolution` can be found in `library/problems/knapsack.py`